# Error analysis


## First import libraries

In [1]:
import numpy as np
%matplotlib inline
import numpy as np
import sys
sys.path.append('./cerform/')
from coare3 import coare3
from air_specific_humidity import air_humidity_method_qsat26air
from surface_specific_humidity import sea_humidity_method_qsee

# estimate errors for current sensor or fluxsat? chose one

In [2]:
#pick one and comment out the other with #
ob_err = 'fluxsat'
#ob_err = 'current'


# define variables

In [22]:
#variable	value	units	notes
Cp = 1004.67         #J/K/kg specific heat at constant pressure
Rgas = 287.1

wind = 15.00000          #m/s    average wind speed over ocean
Tsea = 289.25         #K      average SST over ocean
Tair = Tsea-3        #K      average surface air temperature over ocean
Tsea_degC = Tsea - 273.15
Tair_degC = Tair - 273.15
rel_hum = 0.80000       #N/A    near surface relative humidity
pres = 1013.00000       #mb     average surface pressure used in Bentamy

#calculate Qsea and Qair from Tsea, pres, Tair, rel_hum from coare
Lv = (2.501-0.00237*Tsea)*1e6 #J/kg   latent heat of vaporization  SB had 2500000 CG changed to coare3 def
sat_vap_pres =  6.1121*np.exp(17.502*Tair_degC/(Tair_degC+240.97))*(1.0007+3.46e-6*pres) #from coare3
Qsea = sea_humidity_method_qsee(Tsea_degC,pres)/1000
Qair = air_humidity_method_qsat26air(Tair_degC,pres,rel_hum*100)/1000
air_density = pres*100./(Rgas*(Tair)*(1.+0.61*Qair))

#cH_over_cE = 0.00120    #N/A    aerodynamic transfer coef
#Qsea=18.0/1000
#Qair=15.0/1000
#Qsea=18.0 #/1000 = .0018
#Qair=15.0       = .0015
#rh=80.0
print('Qair',Qair*1000)
print('Qsea',Qsea*1000)


Qair 7.467295302531722
Qsea 11.126587150389046


#  Observational error for either FluxSat or Current variables
- errors from FluxSat via Shannon's simulations
- errors from current variables from publications cited in paper


In [23]:
if ob_err == 'fluxsat':
    dTsea = 0.65 #45
    dwind = 0.6
    dTair = 0.7
    dQair = .6
    dQair_computed = Qair * 0.07 #percent from shannon
    dpres = 5
if ob_err == 'current':
    dTsea = 0.5 #ref found
    dwind = 0.8 #ref found
    dTair = 1.42
    dQair = 1.25
    dQair_computed = 1.25/1000.
    dpres = 5

dpres_computed = (pres+dpres)*100 / (287*Tair) - air_density
print('Qair error:', dQair_computed*1000)

Qair error: 0.5227106711772206


In [24]:
#compute Qsea errors from coare3 using uncertainty in SST & pres
Qsea0 = sea_humidity_method_qsee(Tsea_degC,pres)
Qsea1 = sea_humidity_method_qsee(Tsea_degC+dTsea,pres)
Qsea2 = sea_humidity_method_qsee(Tsea_degC-dTsea,pres)
Qsea_dTsea = (np.maximum(np.abs(Qsea0-Qsea2),np.abs(Qsea0-Qsea1)))
Qsea0 = sea_humidity_method_qsee(Tsea_degC,pres)
Qsea1 = sea_humidity_method_qsee(Tsea_degC,pres+dpres)
Qsea2 = sea_humidity_method_qsee(Tsea_degC,pres-dpres)
Qsea_dpres = (np.maximum(np.abs(Qsea0-Qsea2),np.abs(Qsea0-Qsea1)))
dQsea_computed = np.sqrt(Qsea_dTsea**2+Qsea_dpres**2)/1000
print('Qsea and Qsea uncertainty:',Qsea*1000,dQsea_computed*1000)

Qsea and Qsea uncertainty: 11.126587150389046 0.4765896900489459


# Error propagation for each variable (Shannon Brown)

In [25]:
# calculate Ch and Ce using coare
inputs = {'u':wind,'t':Tair_degC,'Q':Qair*1000,'Qs':Qsea*1000,'P':pres,'ts':Tsea_degC,'lat':35.}
res = coare3(inputs)
Ch = res['Ch']
Ce = res['Ce']
print(Ch,Ce)
print(res['hsb'],res['hlb'])


[0.0012057963794128259] [0.0012057963794128257]
[64.95217820484679] [200.77362178140808]


# Compute Latent Heat Flux errors

In [26]:
dLH_over_dqs =   air_density * Ce * wind * Lv               * dQsea_computed
dLH_over_dqair = air_density * Ce * wind * Lv               * dQair_computed
dLH_over_dwind = air_density * Ce *        Lv * (Qsea-Qair) * dwind
dLH_over_dpres =               Ce * wind * Lv * (Qsea-Qair) * dpres_computed

total_error = np.sqrt(dLH_over_dqs**2+
                      dLH_over_dqair**2+
                      dLH_over_dwind**2+
                      dLH_over_dpres**2)

signal = air_density * Lv * Ce * (Qsea-Qair) * wind

percent_error = total_error / signal *100

print('dLH_over_dqs',dLH_over_dqs)
print('dLH_over_dqair',dLH_over_dqair)
print('dLH_over_dwind',dLH_over_dwind)
print('dLH_over_dpres',dLH_over_dpres)
print('Latent Heat Flux total_error',total_error)
print('signal',signal)
print('percent error',percent_error)

latent_heat_flux_total_error = total_error

dLH_over_dqs [19.2024996]
dLH_over_dqair [21.06078177]
dLH_over_dwind [5.89753003]
dLH_over_dpres [1.4544951639777584]
Latent Heat Flux total_error [29.14084652687554]
signal [147.4382508]
percent error [19.764780420533185]


# Compute Sensible Heat Flux errors

In [27]:
dSH_over_dTsea = air_density * Cp * Ch * wind               * dTsea
dSH_over_dTair = air_density * Cp * Ch * wind               * dTair
dSH_over_dwind = air_density * Cp * Ch        * (Tsea-Tair) * dwind
dSH_over_dpres =               Cp * Ch * wind * (Tsea-Tair) * dpres_computed

total_error = np.sqrt(dSH_over_dTsea**2+
                      dSH_over_dTair**2+
                      dSH_over_dwind**2+
                      dSH_over_dpres**2)

signal = air_density * Cp * Ch * (Tsea-Tair) * wind

percent_error = total_error / signal *100

print('dSH_over_dTsea',dSH_over_dTsea)
print('dSH_over_dTair',dSH_over_dTair)
print('dSH_over_dwind',dSH_over_dwind)
print('dSH_over_dpres',dSH_over_dpres)
print('Sensible Heat Flux total_error',total_error)
#print('signal',signal)
#print('percent error',percent_error)

sensible_heat_flux_total_error = total_error

dSH_over_dTsea [14.49302501]
dSH_over_dTair [15.60787309]
dSH_over_dwind [2.67563539]
dSH_over_dpres [0.6598862080386788]
Sensible Heat Flux total_error [21.476683884366025]
